In [ ]:
import numpy as np
import pandas as pd
from mxlpy import Simulator, plot, unwrap
from mxlpy.integrators import Assimulo

from mxlbricks import get_yokota1985 as get_model
from mxlbricks import names as n

EPS = 1e-6

model = get_model()
res = unwrap(Simulator(model, integrator=Assimulo).simulate(100).get_result())

fig, ax = plot.lines(
    res.get_variables(include_derived_variables=False, include_readouts=False)
)
ax.legend(loc="upper left", bbox_to_anchor=(1, 1), ncols=2)
plot.show()

if (
    diff := np.linalg.norm(
        pd.Series(model.get_initial_conditions()) - pd.Series(res.get_new_y0()), ord=2
    )
) > EPS:
    msg = f"Diff between models too large: {diff}"
    raise ValueError(msg)

In [ ]:
c, v = unwrap(
    Simulator(
        get_model(),
        y0={
            n.glycolate(): 0,
            n.glyoxylate(): 0,
            n.glycine(): 0,
            n.serine(): 0,
            n.hydroxypyruvate(): 0,
            n.h2o2(): 0,
        },
    )
    .simulate(3)
    .get_result()
)
fig, axs = plot.lines_grouped(
    [
        c.loc[:, [n.glycolate(), n.glyoxylate(), n.glycine()]],
        c.loc[:, [n.serine(), n.hydroxypyruvate()]],
        c.loc[:, [n.h2o2()]],
    ],
    n_cols=3,
    col_width=3,
    row_height=2.5,
)
for ax in axs:
    ax.set(xlabel="Time / h", ylabel="Conc. / mM")
plot.show()

fig, axs = plot.lines_grouped(
    [
        v.loc[:, ["phosphoglycolate_phosphatase"]],
        v.loc[:, ["glycine_decarboxylase"]],
        v.loc[:, ["catalase"]],
        v.loc[:, ["glycine_transaminase"]],
    ],
    n_cols=4,
    col_width=3,
    row_height=2.5,
    sharey=False,
)
fig.suptitle("Wildtype")
for ax in axs:
    ax.set(xlabel="Time / h", ylabel="Flux / (mM / h)")
plot.show()